# 은행 고객 예금 가입 예측: 고급 분석 실습 (불균형 처리 · 누수 점검 · 학생용 TODO ONLY)
데이터: bank-additional-full.csv
작성시각: 2025-10-29 07:45

이 노트북은 실제 은행 텔레마케팅 캠페인 데이터를 바탕으로,
EDA → 전처리 → 데이터 불균형 처리(SMOTE 등) → duration 누수 확인 → 모델 학습/평가 → 비즈니스 리포트
엔드투엔드 파이프라인 전체를 직접 구현하는 실습입니다.

중요:
- 각 코드 셀에는 'TODO:'만 있고 실제 코드는 없습니다.
- 모든 분석/시각화/학습/평가는 직접 작성해야 합니다.


## 1. 문제 정의 / 비즈니스 맥락
- 은행은 텔레마케팅(전화 캠페인)으로 정기예금 상품을 판매하려고 한다.
- 타깃 변수 `y`: 'yes' = 실제로 가입했다, 'no' = 가입 안 했다.
- 'yes' 비율은 매우 낮다 → 클래스 불균형(imbalanced data) 상황이다.
- 콜센터 인력/시간은 비용이 크다.
  → "누구에게 우선 전화할 것인가?"를 예측 모델로 결정하고 싶다.
- 이 문제는 단순 정확도(accuracy)보다
  - 가입 가능성이 있는 고객을 최대한 놓치지 않는 능력(recall),
  - 헛콜을 줄이는 능력(precision),
  - 전체 분류력(ROC-AUC)
  같은 지표가 더 중요할 수 있다.


## 2. 데이터 로드 & 기본 개요
목표:
- CSV 데이터를 읽어 DataFrame으로 준비한다.
- 데이터 형태(shape), 컬럼명, 타입, 기본 통계 요약을 파악한다.

필수 확인 포인트:
- 어떤 컬럼이 숫자형인지, 문자열(범주형)인지
- 타깃 변수 `y`의 값 분포
- 각 특성(feature)의 스케일(범위)


In [ ]:

# TODO:
# 1. bank-additional-full.csv 파일을 ';' 구분자로 읽어서 df 라는 이름의 DataFrame으로 저장하세요.
# 2. df.shape 와 df.head()를 출력해서 행/열 수와 데이터 샘플을 확인하세요.
# 3. df.info()를 호출해서 컬럼별 데이터 타입과 결측치 여부를 확인하세요.
# 4. df.describe(include='all').T 를 출력해서 기본 통계 요약(숫자형/범주형 모두)을 확인하세요.


## 3. 데이터 품질 점검 (결측치 / 중복 / 특이값 / 클래스 비율)
목표:
- 결측치가 있는가?
- 중복된 행이 있는가?
- 타깃 y의 클래스 비율은 어느 정도로 불균형인가?
- 특정 컬럼이 이상한 코드값(예: 999)을 쓰고 있는가?
- duration(통화 길이)처럼 극단적으로 치우친 값이 있는가?

힌트:
- `pdays` 컬럼은 999라는 특이값을 많이 가지는데,
  이건 '이전에 캠페인으로 연락한 적 없음'이라는 의미로 쓰인다.
- `duration`은 실제 통화가 얼마나 길었는지(초). 이 값은 "통화가 끝난 뒤"에만 알 수 있음.


In [ ]:

# TODO:
# 1. 각 컬럼별 결측치 개수를 계산하고 내림차순으로 출력하세요.
#    (df.isna().sum().sort_values(ascending=False))
#
# 2. df.duplicated().sum() 으로 중복 행이 몇 개인지 출력하세요.
#
# 3. 타깃 y 의 분포를 value_counts(), value_counts(normalize=True)로 확인하세요.
#    그리고 막대그래프로 시각화하세요 (y가 yes/no 중 어느 쪽이 얼마나 많은지).
#
# 4. pdays 컬럼의 상위 value_counts()를 출력해보고 999가 의미하는 바를 주석으로 적어보세요.
#
# 5. duration 컬럼에 대해 describe()를 출력하고,
#    최소/최대/평균 등을 바탕으로 이상치(극단적 길이)가 존재하는지 메모하세요.


## 4. EDA (탐색적 분석)
이 단계에서는
- 숫자형/범주형 컬럼을 자동으로 분리하고
- 분포, 관계, 상관성을 시각적으로 점검한다.

4.1 숫자형/범주형 컬럼 자동 분리  
4.2 숫자형 분포(히스토그램)  
4.3 범주형 분포(막대그래프) 및 y별 비교  
4.4 y와 연속형 변수 관계(boxplot)  
4.5 상관관계 히트맵(수치형 vs y)


In [ ]:

# TODO:
# 1. get_numeric_columns(df), get_categorical_columns(df) 라는 함수를 직접 정의하세요.
#    - 숫자형 컬럼 목록만 반환
#    - 범주형(문자형 등) 컬럼 목록만 반환
#
# 2. numeric_cols, categorical_cols 변수에 그 결과를 담고 출력하세요.


In [ ]:

# TODO:
# 숫자형 컬럼(numeric_cols)에 대한 히스토그램을 그리세요.
# 힌트: df[numeric_cols].hist(figsize=(12,8), bins=20) 형태로 전체 분포를 한 번에 볼 수 있습니다.
# 각 컬럼마다 값의 분포가 치우쳐 있는지(스큐), 극단값이 있는지 메모하세요.


In [ ]:

# TODO:
# 범주형 컬럼(categorical_cols)의 값 분포를 막대그래프로 그리세요.
# - 반복문으로 각 col마다 value_counts() 상위 순서대로 막대그래프(countplot 등)를 그리고
#   x tick을 회전해서 보이게 하세요.
#
# 추가로 예시:
# - 특정 범주형 변수(예: job)에 대해 hue='y' 로 countplot을 그려서
#   어떤 직업군에서 'yes'가 상대적으로 더 많이 나오는지 확인하세요.
#   (= 캠페인 타깃팅 힌트)


In [ ]:

# TODO:
# boxplot을 사용해 y에 따라 age 분포가 어떻게 다른지 그리세요.
# boxplot을 사용해 y에 따라 duration 분포가 어떻게 다른지도 그리세요.
#
# 관찰한 점을 마크다운 셀로 기록하세요:
# 예) "가입한 사람(yes)은 duration이 전반적으로 더 길다" 같은 해석.


In [ ]:

# TODO:
# 1. df.copy()로 복사한 후 y를 0/1로 변환한 새 컬럼 (예: y_num) 을 만드세요.
#    예: 'yes' -> 1, 'no' -> 0
#
# 2. numeric_cols + ['y_num'] 로 상관계수 행렬을 구하세요.
#
# 3. heatmap 으로 시각화하세요.
#    어떤 수치형 변수가 y_num과 양(+)의 상관/음(-)의 상관을 가지는지 확인하고
#    그 의미를 간단히 적어보세요.


## 5. 전처리 / 파생변수 / 인코딩 / 데이터셋 분리 / 스케일링
목표:
- y를 0/1 숫자로 변환
- 비즈니스 룰 기반 파생변수 만들기 (예: 이전 캠페인에서 연락한 적 없는지 여부)
- duration은 실제 통화가 끝난 뒤에만 알 수 있으므로, 실제 배포 모델에서는 쓰지 않는 버전도 함께 준비
- 범주형 변수를 더미변수로 변환 (pd.get_dummies)
- 학습/평가용 train/test 분리 (층화 stratify 사용)
- 숫자형 컬럼만 StandardScaler로 스케일링해서 로지스틱 회귀에 투입할 준비


In [ ]:

# TODO:
# 1. 타깃 y를 0/1로 변환한 Series를 만드세요. (예: {'no':0, 'yes':1} 매핑)
#
# 2. 특징행렬 X_full 을 df에서 y만 제거한 형태로 만들고 복사하세요.
#
# 3. 'pdays == 999' 여부를 나타내는 파생변수 no_prev_contact 를 X_full에 추가하세요.
#    이건 '이전 캠페인에서 연락한 적 없음'이라는 비즈니스 신호입니다.
#
# 4. duration 은 통화가 끝나야만 알 수 있는 정보입니다.
#    실제 캠페인 사전 추천 모델에는 넣을 수 없으므로,
#    duration 을 뺀 버전 X_nodur 를 따로 만드세요.
#
# 5. 두 버전에 대해 pd.get_dummies(drop_first=True)로 범주형 원-핫 인코딩을 하세요.
#    예: X_full_enc, X_nodur_enc
#
# 6. train_test_split을 사용해서 각 버전을 학습/테스트(예: 80/20)로 나누세요.
#    stratify=y 를 꼭 사용하세요. (불균형 유지)
#    예: Xf_train, Xf_test, yf_train, yf_test
#        Xn_train, Xn_test, yn_train, yn_test
#
# 7. StandardScaler를 사용해 숫자형 컬럼만 스케일링한 학습/테스트 버전을 만드세요.
#    - WITH duration 용 (Xf_train_scaled / Xf_test_scaled)
#    - NO duration 용  (Xn_train_scaled / Xn_test_scaled)
#
# 8. 왜 duration 특징이 실제 배포 모델에 들어가면 안 되는지
#    마크다운 셀로 설명하세요.
#    키워드: "데이터 누수(data leakage)", "미래 정보"


## 6. 데이터 불균형 처리 (SMOTE / RandomOverSampler / RandomUnderSampler)
목표:
- 학습 데이터에서 'yes' 클래스(가입 고객)가 매우 적은 문제를 완화한다.
- Over-sampling: 소수 클래스를 늘린다.
  - RandomOverSampler: 단순 복제
  - SMOTE: 인공적으로 합성 샘플 생성
- Under-sampling: 다수 클래스를 줄인다.
  - RandomUnderSampler
- resampling 전과 후의 클래스 분포를 비교하고 시각화한다.


In [ ]:

# TODO:
# 1. 현재 학습 데이터(예: Xn_train / yn_train 처럼 duration 제거 버전의 train 세트)의
#    타깃 클래스 분포를 bar 플롯으로 시각화하세요.
#
# 2. RandomOverSampler로 오버샘플링:
#    - X_ros, y_ros 를 만들고
#    - y_ros의 클래스 분포를 출력하세요.
#
# 3. SMOTE로 오버샘플링:
#    - X_smote, y_smote 를 만들고
#    - y_smote의 클래스 분포를 출력하세요.
#
# 4. RandomUnderSampler로 언더샘플링:
#    - X_rus, y_rus 를 만들고
#    - y_rus의 클래스 분포를 출력하세요.
#
# 5. 원본 vs SMOTE 결과 분포를 나란히 bar 차트로 그리고 비교하세요.
#
# 6. 마크다운 셀에 장단점을 적으세요:
#    - Over-sampling의 장점/단점
#    - Under-sampling의 장점/단점
#    - 어떤 상황에서 어느 쪽이 더 적합하다고 생각하는지


## 7. 모델 학습 / 비교
목표:<br>
- 서로 다른 데이터/전처리 조건에서 모델을 학습시키고 비교한다.<br>

아래 네 가지(최소)를 직접 학습/예측하세요:<br>

1. Logistic Regression<br>
   1-1. WITH duration (스케일된 Xf_train_scaled 사용, yf_train으로 학습)<br>
   1-2. NO duration (스케일된 Xn_train_scaled 사용, yn_train으로 학습)<br>
   1-3. NO duration + SMOTE (SMOTE로 만든 X_smote / y_smote 사용,<br>
        이 데이터는 따로 스케일링해서 학습해야 함)<br>

2. RandomForestClassifier<br>
   2-1. NO duration (Xn_train / yn_train 사용)<br>
   2-2. NO duration + SMOTE (X_smote / y_smote 사용)<br>
        → 랜덤포레스트도 불균형 개선 효과를 비교해본다.


In [ ]:

# TODO:
# (1) 로지스틱 회귀 LogisticRegression
#   a. WITH duration :
#      - Xf_train_scaled / yf_train 으로 학습
#      - Xf_test_scaled 로 예측 -> ypred_log_full, yprob_log_full
#
#   b. NO duration :
#      - Xn_train_scaled / yn_train 으로 학습
#      - Xn_test_scaled 로 예측 -> ypred_log_nodur, yprob_log_nodur
#
#   c. NO duration + SMOTE :
#      - 먼저 X_smote / y_smote 에 대해 숫자형 컬럼만 다시 스케일링
#        (train 기준으로 fit, test는 Xn_test 기준 transform)
#      - 예측 -> ypred_log_smote, yprob_log_smote
#
# (2) 랜덤포레스트 RandomForestClassifier
#   d. RF NO duration :
#      - Xn_train / yn_train 으로 학습
#      - Xn_test 로 예측 -> ypred_rf_nodur, yprob_rf_nodur
#
#   e. RF NO duration + SMOTE :
#      - X_smote / y_smote 로 학습
#      - Xn_test 로 예측 -> ypred_rf_smote, yprob_rf_smote
#
# 각 모델마다 class_weight='balanced' 또는 'balanced_subsample' 같은 옵션을 사용해서
# 불균형 보정도 시도해보세요.
#
# 중요: duration 을 포함한 모델은 실제 배포 가능한 모델인가요? 아니면 누수(leakage) 때문에 금지인가요?
#      마크다운 셀에 설명하세요.


## 8. 평가 / 비교 (classification_report, ROC-AUC, ROC Curve)
목표:
- 각 모델의 성능을 서로 비교한다.
- 단순 accuracy가 아니라, recall / precision / f1-score / ROC-AUC 같은 지표를 집중해서 본다.
- ROC Curve를 한 그래프에 겹쳐서 시각화하고 차이를 해석한다.

여기서는 로지스틱 회귀뿐 아니라 랜덤포레스트도
SMOTE 전/후 버전을 비교해서 시각화해야 한다.


In [ ]:

# TODO:
# 1. classification_report를 이용해 각 모델의 정밀도(precision), 재현율(recall),
#    f1-score를 출력하세요.
#
#    예시로 비교해야 하는 모델들:
#    - LogReg WITH duration
#    - LogReg NO duration
#    - LogReg NO duration + SMOTE
#    - RF NO duration
#    - RF NO duration + SMOTE
#
# 2. roc_auc_score 로 각 모델의 ROC-AUC를 계산하세요.
#
# 3. roc_curve 를 사용해
#    (a) 로지스틱 3개 (WITH duration / NO duration / NO duration+SMOTE)
#    (b) 랜덤포레스트 2개 (NO duration / NO duration+SMOTE)
#    각각 ROC 곡선을 그리고, AUC 값을 범례 라벨에 넣으세요.
#
#    즉, ROC 그래프를 2장 그려도 됩니다:
#    - 그래프1: 로지스틱 3종 비교
#    - 그래프2: 랜덤포레스트 2종 비교
#
# 4. 어떤 모델이 "현실 배포 가능한 모델"인지, 그 이유를 마크다운 셀에 작성하세요.
#    (duration이 있는 모델은 왜 배포 불가인가?)
#
# 5. SMOTE가 특히 어느 모델에서(로지스틱 vs 랜덤포레스트) 더 큰 이득을 줬는지,
#    그 근거를 적으세요. (recall 향상? AUC 향상?)


## 9. 리포트 질문 (마크다운 셀에 직접 서술)
Q1. duration(통화 길이) 변수를 학습에 포함하면 왜 안 될까요?
    - '데이터 누수(data leakage)'라는 표현을 포함해서 3줄 이상으로 설명하세요.

Q2. SMOTE처럼 소수 클래스(가입 고객)를 증폭시키는 방식은 어떤 장점과 위험을 동시에 가지고 있나요?
    - recall, precision 관점에서 설명해보세요. (3줄 이상)

Q3. 실제 은행 마케팅 KPI(콜 인력 비용, 고객 피로도, 브랜드 이미지 등)를 생각했을 때
    어떤 지표를 핵심 지표로 선택하겠습니까?
    - recall(가입할 고객을 놓치지 않기) vs precision(헛콜 줄이기) vs F1 vs ROC-AUC
    중 하나를 골라서 4줄 이상으로 설득력 있게 적어보세요.
